In [1]:
import MeCab
import unidic
print(unidic.DICDIR)
MeCab.Tagger("-d" + unidic.DICDIR)

/home/yamada/.local/lib/python3.6/site-packages/unidic/dicdir


<Swig Object of type 'MeCab::Tagger *' at 0x7f690827c6d0>

- よくわからないもの

    
    

In [2]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [3]:
ngram_path = "../../corpus/google/3-gram/"

In [4]:
key1_set = set( os.listdir(ngram_path) ) 

In [17]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

error = "Grammatical error"
# errors = ['Grammatical error', "Uninterpretable"]
errors = ['Semantic error', "Wrong information"]
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            if not ut.utt[-1] in ["？", "！", "。", "!"]:
                sys_utt.append( clean_text( ut.utt+"。" ))
                # sys_utt.append(ut.utt)
            else:   
                sys_utt.append(ut.utt)
            if ut.is_error_included(errors):
                y.append(1)
            else:
                y.append(0)

In [6]:
print("len of y:{0}, error '{1}' counts:{2}".format(len(y), errors, y.count(1)))

len of y:1386, error '['Grammatical error', 'Uninterpretable']' counts:7


In [7]:
def load_dict(path, key1):
    filename = key_formated(key1)
    with open(path+filename, "r") as f:
        key1_dict = json.load(f) 
    return key1_dict  

In [8]:
def key_formated(key1):
    return "ngram_"+key1+".json"

In [9]:
key2dict = dict()
registerform_dict = dict()
analyzer_unidic=  Analyzer(Tokenizer())

In [10]:
#  を考慮
# 何故か↑が出てくる
def judge_key3(key1_dict, ngram):
    key1 = ngram[0]
    key2 = ngram[1]
    key3 = ngram[2]

    if key2 not in key1_dict[key1]:
        # '"'をkeyにして聞いてみる
        if '"' in key1_dict[key1] and key2 in key1_dict[key1]['"']:
            return True
        if '、' in key1_dict[key1] and key2 in key1_dict[key1]['、']:
            return True
        return False
    
    if key3 not in key1_dict[key1][key2]:
        return False

    return True 

In [11]:

def search_registerforam(key, reading):
    filename = key_formated(key)
    if filename in key1_set:
        return key
    # 登録されていない
    else:
        for other in reading:
            filename = key_formated(key)
            if filename in key1_set:
                registerform_dict[key] = other
                return other
        return key


def sentence_judge(sentence, N=3):
    
    df = analyzer_unidic.analyze_with_dataframe(sentence)
    
    # 登録
    sentence_ = ""
    for txt, read in zip(df.surface, df.reading):
        sentence_ += search_registerforam(txt, read)
    sents = sentence2docs(sentence_, sents_span=True)
    # print(sents)

    # print(tokens)
    ngram_text = []
    ngram_pos = []

    for sent in sents:
        # 各文章 : sent
        df = analyzer_unidic.analyze_with_dataframe("".join([token.orth_ for token in sent]))
        surfaces = []
        poses = []
        readings = []
        for txt, p, read in zip(df.surface, df.part_of_speech, df.reading):
            surfaces.append(txt)
            readings.append(read)
            poses.append(p)

            for i in range(len(surfaces)-N+1):
                # print(L[i:i+N])
                ngram_text.append(surfaces[i:i+N])
                ngram_pos.append(poses[i:i+N])
                # ngram_reading.append(readings[i:i+N])
    
    # print(ngram_text)
    # return
    
    for ngram, pgram in zip(ngram_text, ngram_pos):
        # if "助動詞" in pgram[1] or "助詞" in pgram[1] or "助動詞" in pgram[0] or "助詞" in pgram[0]:
        # if "助動詞" in pgram[0] or "助詞" in pgram[0]:
        if "助詞" in pgram[1]:
            # print("\tcontext : {0}| ->".format(ngram))

            key1 = ngram[0]
            if key_formated(key1) not in key1_set:
                continue

            if key1 in key2dict:
                key1_dict = key2dict[key1]
            else:
                key1_dict = load_dict(ngram_path, key1)
                key2dict[key1] = key1_dict

            # チェック
            if not judge_key3(key1_dict, ngram):
                print(ngram)
                # reading を調整
                
                # 見つかれば，その語彙を登録
                return False
    return True
            


In [12]:
y_pred_2 = []
for i, utt in enumerate( tqdm( sys_utt ) )  :
    cleaned = clean_text(utt).replace("?", "。").replace("!", "")
    # print(cleaned)
    if sentence_judge(cleaned, N=3):
        y_pred_2.append(0)
    else:
        y_pred_2.append(1)
    # if i==35:
    #     break

  0%|          | 2/1386 [00:00<08:18,  2.78it/s]

['遍路', 'の', '逆回り']


  4%|▍         | 55/1386 [00:20<03:04,  7.20it/s] 

['天麩羅', 'は', '食べる']
['寿司', 'は', '縁側']
['なん', 'と', '。']


  5%|▌         | 70/1386 [00:25<03:51,  5.68it/s]

...

['因み', 'に', '足']
['因み', 'に', '足']


 99%|█████████▉| 1375/1386 [02:01<00:00, 30.17it/s]

['それ', 'は', '仕方ない']
['作っ', 'て', 'いやいや']


100%|██████████| 1386/1386 [02:01<00:00, 11.40it/s]

['実', 'は', '。']


In [19]:
y_pred_2.count(0)

754

In [20]:
score(y, y_pred_2)

confusion matrix = 
 [[691 292]
 [ 63 340]]
accuracy =  0.7438672438672439
precision =  0.5379746835443038
recall =  0.8436724565756824
f1 score =  0.6570048309178744


- 使い方が難しい

        confusion matrix = 
        [[666 713]
        [  6   1]]
        accuracy =  0.48124098124098125
        precision =  0.0014005602240896359
        recall =  0.14285714285714285
        f1 score =  0.0027739251040221915

- ちょっと改善(プログラムミスを修正)

        confusion matrix = 
        [[870 509]
        [  6   1]]
        accuracy =  0.6284271284271284
        precision =  0.00196078431372549
        recall =  0.14285714285714285
        f1 score =  0.0038684719535783366

    - 意外と普通っぽい ngram がない
        - ["を", "覚える", *]

    - スコア化はせず， ngram が存在しなければアウトと仮定した
        - 先行研究のモデルで，そういったボーダーは存在しない

- 用法エラー + 誤情報

        confusion matrix = 
        [[758 225]
        [118 285]]
        accuracy =  0.7525252525252525
        precision =  0.5588235294117647
        recall =  0.707196029776675
        f1 score =  0.6243154435925521
    

    

In [21]:
data_path = "../X_y_data/base_y_pred/"
# data_name = "utt_form.pickle"
data_name = "utt_content.pickle"
dataM = DataManager(data_path)
dataM.save_data(data_name, [y, y_pred_2])

success save : ../X_y_data/base_y_pred/utt_content.pickle


In [16]:
df = analyzer_unidic.analyze_with_dataframe("頼りがいのあった可愛くない兄さんが死んだ")
df

,surface,part_of_speech,infl_type,infl_form,base_form,reading,phonetic
0,頼りがい,名詞-普通名詞-一般-,,,タヨリガイ,"[頼り甲斐, 頼りがい, タヨリガイ, 頼りがい, タヨリガイ]",*
1,の,助詞-格助詞--,,,ノ,"[の, の, ノ, の, ノ]",*
2,あっ,動詞-非自立可能--,五段-ラ行,連用形-促音便,アル,"[有る, あっ, アッ, ある, アル]",*
3,た,助動詞---,助動詞-タ,連体形-一般,タ,"[た, た, タ, た, タ]",*
4,可愛く,形容詞-一般--,形容詞,連用形-一般,カワイイ,"[可愛い, 可愛く, カワイク, 可愛い, カワイー]",*
5,ない,形容詞-非自立可能--,形容詞,連体形-一般,ナイ,"[無い, ない, ナイ, ない, ナイ]",*
6,兄,名詞-普通名詞-一般-,,,アニ,"[兄, 兄, アニ, 兄, アニ]",*
7,さん,接尾辞-名詞的-一般-,,,サン,"[さん, さん, サン, さん, サン]",*
8,が,助詞-格助詞--,,,ガ,"[が, が, ガ, が, ガ]",*
9,死ん,動詞-一般--,五段-ナ行,連用形-撥音便,シヌ,"[死ぬ, 死ん, シン, 死ぬ, シヌ]",*
